## Задание 1. (10 баллов)

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning).
Можно использовать alpaca_dataset, датасет Dolly 2 или переведенный датасет (или все вместе).
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning).
Размер модели должен быть как минимум 3 млрд параметров.  
**Нужно использовать модель, которую мы не разбирали на семинаре (OPT-2.7b, OPT-6.7b). Найдите новую модель на huggingface hub.**



In [ ]:
# !pip install -q git+https://github.com/huggingface/transformers.git@main
!pip install transformers==4.31.0
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.0 MB/s eta 0:00:00


In [ ]:
# !wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

--2024-05-15 22:42:00--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M   133MB/s    in 0.2s    

2024-05-15 22:42:00 (133 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



In [ ]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
torch.__version__

'2.2.1+cu121'

In [ ]:
import pandas as pd
import numpy as np
import torch
import json

In [ ]:
# data_alpaca = json.load(open('alpaca_data.json'))
# data_alpaca[:3]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-1.3b',
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're being sarcastic or not, but I'm going to go with sarcastic.


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [ ]:
!nvidia-smi

Wed May 15 23:13:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              50W /  70W |   5217MiB / 15360MiB |     77%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

побоялась сразу брать большую модель, сначала попробуем facebook/opt-1.3b

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",
    quantization_config=quantization_config,
    cache_dir='./models'
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're being sarcastic or not, but I'm going to go with sarcastic.


In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# вот так можно (в torch) напечатать слои и их названия
# в нашем случае на них прямо написано, что они линейные (proj и fc)
# поэтому мы можем выбрать q_proj, v_proj, k_proj, out_proj, fc1 и fc2
for name, module in model.named_modules():
    print(name)


model
model.decoder
model.decoder.embed_tokens
model.decoder.embed_positions
model.decoder.final_layer_norm
model.decoder.layers
model.decoder.layers.0
model.decoder.layers.0.self_attn
model.decoder.layers.0.self_attn.k_proj
model.decoder.layers.0.self_attn.v_proj
model.decoder.layers.0.self_attn.q_proj
model.decoder.layers.0.self_attn.out_proj
model.decoder.layers.0.activation_fn
model.decoder.layers.0.self_attn_layer_norm
model.decoder.layers.0.fc1
model.decoder.layers.0.fc2
model.decoder.layers.0.final_layer_norm
model.decoder.layers.1
model.decoder.layers.1.self_attn
model.decoder.layers.1.self_attn.k_proj
model.decoder.layers.1.self_attn.v_proj
model.decoder.layers.1.self_attn.q_proj
model.decoder.layers.1.self_attn.out_proj
model.decoder.layers.1.activation_fn
model.decoder.layers.1.self_attn_layer_norm
model.decoder.layers.1.fc1
model.decoder.layers.1.fc2
model.decoder.layers.1.final_layer_norm
model.decoder.layers.2
model.decoder.layers.2.self_attn
model.decoder.layers.2.self_

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'out_proj', 'fc1', 'fc2'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 28311552 || all params: 1344069632 || trainable%: 2.106405153866314


tatsu-lab/alpaca

In [ ]:
import transformers
from datasets import load_dataset

# в качестве датасета я взял инструкции к генерации кода
data = load_dataset("tatsu-lab/alpaca", split='train[:20%]')

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)

Map:   0%|          | 0/10400 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

In [ ]:
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to doc

Step,Training Loss
1,2.722300
2,2.612500
3,2.752500
4,2.536700
5,2.577600
6,2.538600
7,2.840800
8,2.697200
9,2.617800
10,2.527000


TrainOutput(global_step=400, training_loss=1.3247316047549247, metrics={'train_runtime': 2093.962, 'train_samples_per_second': 3.056, 'train_steps_per_second': 0.191, 'total_flos': 8221224285634560.0, 'train_loss': 1.3247316047549247, 'epoch': 0.62})

In [ ]:
model.save_pretrained('opt_1.3_lora')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "opt_1.3_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b",
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
print(generate("Give three tips for staying healthy. ",  tokenizer, model))

Give three tips for staying healthy.   1. Eat healthy. 2. Exercise. 3. Sleep.  If you do all three, you'll be healthy.
I'm not sure if I can do all 3 at the same time. I'm not a morning person. I


In [ ]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list!    I have a lot of things to do today.   My todo is to get a new pair of shoes.  I'm not sure what to get.  Any suggestions?   Also, I have to go to the doctor


In [ ]:
print(generate("Tell me a joke. ",  tokenizer, model))

Tell me a joke.    What's the difference between a man and a bag of potatoes?   I can't tell you.
I can't believe you're the only one to get this.


In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
print(generate("Give three tips for staying healthy. ",  tokenizer, model))

Give three tips for staying healthy. 
1\. Eat a healthy diet.
2\. Exercise regularly.
3\. Get enough sleep.

1. Eat a balanced diet. 2. Exercise regularly 3. Get enough rest.
1 is the most important.
If you


In [ ]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list! 
1\. Create a to-do list for the week.
2\. Create an email template for the weekly email.
3\. Create and send out a weekly newsletter.
4\. Create weekly blog posts.
5\. Create monthly blog


In [ ]:
print(generate("Tell me a joke. ",  tokenizer, model))

Tell me a joke. 
What's the difference between a cat and a dog?    One can't jump.
What do you call a cat with no eyes?  A blind cat.
How do you make a cat cry?  Give it a piece


ужас ну и шуточки

### facebook/opt-13b

In [ ]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-13b',
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-13b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/52.4k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.86G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/5.87G [00:00<?, ?B/s]

In [ ]:
# batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
# output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
# print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
!nvidia-smi

In [ ]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-13b",
    quantization_config=quantization_config,
    cache_dir='./models'
)

In [ ]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'out_proj', 'fc1', 'fc2'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

In [ ]:
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
model.save_pretrained('opt_13_lora')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "opt_13_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-13b",
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-13b")

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
print(generate("Give three tips for staying healthy. ",  tokenizer, model))

Give three tips for staying healthy.   1. Eat healthy. 2. Exercise. 3. Sleep.  If you do all three, you'll be healthy.
I'm not sure if I can do all 3 at the same time. I'm not a morning person. I


In [ ]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list!    I have a lot of things to do today.   My todo is to get a new pair of shoes.  I'm not sure what to get.  Any suggestions?   Also, I have to go to the doctor


In [ ]:
print(generate("Tell me a joke. ",  tokenizer, model))

Tell me a joke.    What's the difference between a man and a bag of potatoes?   I can't tell you.
I can't believe you're the only one to get this.


In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
print(generate("Give three tips for staying healthy. ",  tokenizer, model))

Give three tips for staying healthy. 
1\. Eat a healthy diet.
2\. Exercise regularly.
3\. Get enough sleep.

1. Eat a balanced diet. 2. Exercise regularly 3. Get enough rest.
1 is the most important.
If you


In [ ]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list! 
1\. Create a to-do list for the week.
2\. Create an email template for the weekly email.
3\. Create and send out a weekly newsletter.
4\. Create weekly blog posts.
5\. Create monthly blog


In [ ]:
print(generate("Tell me a joke. ",  tokenizer, model))

Tell me a joke. 
What's the difference between a cat and a dog?    One can't jump.
What do you call a cat with no eyes?  A blind cat.
How do you make a cat cry?  Give it a piece
